## Finetuning Mistral на диалоговом датасете

In [ ]:
import json
import torch
from datasets import Dataset
from peft import LoraConfig
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

**Загружаем датасет**

**Получить бесплатный доступ к датасету Company Cases** очень просто:

**1.** Подписываетесь на наши ресурсы: https://vk.com/pine_forest_ai, https://t.me/+Ml16EbQoepcwMGNi, https://www.linkedin.com/company/pineforest-ai/ и https://www.youtube.com/@PineForestAI.

**2.** Подаете заявку в группу https://vk.com/adv_nlp_course, в которой находится датасет.

**3.** Мы в течение нескольких часов принимаем заявку и вы сможете скачать в группе датасет.

In [ ]:
with open("company_cases.json", 'r') as inp:
    raw_dataset = json.load(inp)

train_raw_dataset, test_raw_dataset = train_test_split(raw_dataset)
train_test_raw_dataset = {"train": train_raw_dataset, "test": test_raw_dataset}

**Преобразуем датасет в формат, который используется в классе Dataset**

In [ ]:
raw_dataset_dict = {}
for data_type in ["train", "test"]:
    raw_dataset_dict[data_type] = {
        "instruction": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "personality": [element['personality'] for element in train_test_raw_dataset[data_type]],
        "context": [element['context'] for element in train_test_raw_dataset[data_type]],
        "dialog_start_line": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "dialog": [element['dialog'] for element in train_test_raw_dataset[data_type]]
    }

train_dataset = Dataset.from_dict(raw_dataset_dict["train"])
test_dataset = Dataset.from_dict(raw_dataset_dict["test"])

**Функция для форматирования промпта**

In [ ]:
def formatting_prompts_func(examples):
    prompts = []
    for i in range(len(examples['instruction'])):
        prompt = f"<s>system\n{examples['instruction'][i]}"
        if examples['personality'][i]:
            prompt += f"\n{examples['personality'][i]}"
        prompt += f"\n{examples['context'][i]}"
        if examples['dialog_start_line']:
            prompt += f"\n{examples['dialog_start_line'][i]}"
        prompt += "</s>"
        for element in examples['dialog'][i]:
            prompt += f"<s>{element['role']}\n{element['content']}</s>"
        prompts.append(prompt)
    return prompts

**Инициализируем токенизатор**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca")
tokenizer.pad_token_id = 0

Сделаем, чтобы **градиент при обучении протекал только через токены последней реплики чат-бота**, labels у остальных токенов сделаем равными -100.

In [ ]:
response_template = "bot\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

**Инициализируем модель**

In [ ]:
# Будем обучать модель в int4 для уменьшения требуемой видеопамяти

model = AutoModelForCausalLM.from_pretrained(
    "Open-Orca/Mistral-7B-OpenOrca",
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4', # квантизация модели в тип normal float 4
    )
)

**Параметры адаптеров**: $r$ - размерность матрицы адаптеров, $lora\_alpha$ - это значение, на которое после умножения вектора на матрицу-адаптер мы будем умножать каждую компоненту полученного вектора.

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=100,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    warmup_steps=100,
    save_steps=100,
    eval_steps=100,
    save_total_limit=3
)

**Инициализируем SFTTrainer и запускаем обучение.**

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    peft_config=peft_config,
    args=training_arguments,
    max_seq_length=3700
)

trainer.train()